In [6]:
import keras
from keras.optimizers import Adam
from Score.helpers import *
from Score.algorithms import trustworthiness
tf.compat.v1.disable_eager_execution()

In [7]:
# Todo: Specify evaluation scenario & limited
scenario = "creditrisk" # diabetes or creditrisk
limited = False # if True, computes metrics that do not depend on DS

In [8]:
# ********** Load respective model and dataset ************
if scenario == "diabetes":
        solution = "Solutions/S1DiabetesPredictionRF/TrueScore"
        model = pickle.load(open(f"{solution}/model.pkl", "rb")) # RF
        target_column = "diabetes"
elif scenario == "creditrisk":
        solution = "Solutions/S2HomeCreditDefaultRiskDNN/TrueScore"
        model = keras.models.load_model(f"{solution}/model.keras") # DNN
        optimizer = Adam(learning_rate=0.001)
        model.compile(loss="categorical_crossentropy", optimizer=optimizer)
        target_column = "TARGET"
else:
        raise AssertionError("Check specifications: Evaluation scenario does not exist")
if limited:
        train = None
        test = None
        solution = f"{'/'.join(solution.split('/')[:-1])}/LimitedScore"
else:
        train = pd.read_csv(f"{solution}/train.csv")
        test = pd.read_csv(f"{solution}/test.csv")

In [ ]:
# ********** Compute TRUE Score ************
factsheet = json.load(open(f"{solution}/factsheet.json"))
config_fairness = json.load(open(f"Score/configs/mappings/fairness/default.json"))
config_exp = json.load(open(f"Score/configs/mappings/explainability/default.json"))
config_rob = json.load(open(f"Score/configs/mappings/robustness/default.json"))
config_met = json.load(open(f"Score/configs/mappings/methodology/default.json"))
config_weights = json.load(open(f"Score/configs/weights/default.json"))
solution_set_path = f"{solution}/"
mappings_config = {}
mappings_config["fairness"] = config_fairness
mappings_config["explainability"] = config_exp
mappings_config["robustness"] = config_rob
mappings_config["methodology"] = config_met
result = trustworthiness.get_final_score(model, train, test, config_weights, mappings_config, factsheet, solution_set_path, recalc=False)
true_final_scores, true_results, true_properties = result

In [ ]:
# ********** Plotting for Report ************
from Score.plot import *

trust_score = trustworthiness.get_trust_score(true_final_scores, config_weights["pillars"])
data = {"final_score":true_final_scores,
        "results":true_results,
        "trust_score":trust_score,
        "properties" : true_properties}

final_score = data["final_score"]
pillars = list(final_score.keys())
values = list(final_score.values())
pillar_colors = ['#06d6a0', '#ffd166', '#ef476f', '#118ab2']
title = "Overall Trust Score {}/5".format(trust_score)
figname = "overall.png"

draw_bar_plot(pillars, values, pillar_colors, title, figname, solution)
draw_pillar_scores(data['results'], final_score, pillar_colors, solution)